### Singleton Pattern

- Sometimes you need an object in an application where there is only one instance.
- You don't want there to be many versions, for example, you have a game with a score, and you want to adjust it. You may have accidentally created several instances of the class holding the score object. Or, you may be opening a database connection, there is no need to create many, when you can use the existing one that is already in memory. You may want a logging component, and you want to ensure all classes use the same instance. So, every class could declare their own logger component, but behind the scenes, they all point to the same memory address (ID).
- By creating a class and following the Singleton pattern, you can enforce that even if any number of instances were created, they will still refer to the original class.
- The Singleton can be accessible globally, but it is not a global variable. It is a class that can be instanced at any time, but after it is first instanced, any new instances will point to the same instance as the first.
- For a class to behave as a Singleton, it should not contain any references to self but use static variables, static methods and/or class methods.

In [1]:
from abc import ABCMeta, abstractmethod, abstractstaticmethod

In [ ]:
!pip install sqlalchemy python-dotenv "fastapi[standard]" pymysql

  Using cached sqlalchemy-2.0.44-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.5 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached fastapi-0.121.0-py3-none-any.whl.metadata (28 kB)
  Using cached greenlet-3.2.4-cp313-cp313-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (4.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached starlette-0.49.3-py3-none-any.whl.metadata (6.4 kB)
  Using cached annotated_doc-0.0.3-py3-none-any.whl.metadata (6.6 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached sqlalchemy-2.0.44-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
Using

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
from threading import Lock
from fastapi import Depends
from dotenv import load_dotenv
import os
import pymysql

# Load environment variables
load_dotenv()

pymysql.install_as_MySQLdb()

class Database:
    """Singleton Database connection for SQLAlchemy."""
    _instance = None
    _lock = Lock()

    def __new__(cls):
        if cls._instance is None:
            with cls._lock:  # thread-safe
                if cls._instance is None:
                    cls._instance = super(Database, cls).__new__(cls)
                    cls._instance._init_engine()
        return cls._instance

    def _init_engine(self):
        DATABASE_DRIVER = os.getenv("DATABASE_DRIVER")
        DATABASE_USER = os.getenv("DATABASE_USER")
        DATABASE_PASSWORD = os.getenv("DATABASE_PASSWORD")
        DATABASE_HOST = os.getenv("DATABASE_HOST")
        DATABASE_PORT = os.getenv("DATABASE_PORT")
        DATABASE_NAME = os.getenv("DATABASE_NAME")

        self.SQLALCHEMY_DATABASE_URL = (
            f"{DATABASE_DRIVER}://{DATABASE_USER}:{DATABASE_PASSWORD}@"
            f"{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"
        )

        self.engine = create_engine(self.SQLALCHEMY_DATABASE_URL, pool_pre_ping=True)
        self.SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=self.engine)
        self.Base = declarative_base()
        print("✅ Database engine initialized (Singleton)")

    def get_db(self):
        """Provide a new SQLAlchemy session (dependency for FastAPI)."""
        db = self.SessionLocal()
        try:
            yield db
        finally:
            db.close()


# Dependency to use in FastAPI endpoints
def get_db() -> Session:
    db_instance = Database()
    return next(db_instance.get_db())


- In the example, there are three games created. They are all independent instances created from their own class, but they all share the same leaderboard. The leaderboard is a singleton.
- It doesn't matter how the Games where created, or how they reference the leaderboard, it is always a singleton.
- Each game independently adds a winner, and all games can read the altered leaderboard regardless of which game updated it.

In [5]:
class IGame(metaclass=ABCMeta):
    @abstractmethod
    def add_winner(self, name: str):
        pass
    
    @abstractmethod
    def get_leaderboard(self):
        pass
    

class LeaderBoard():
    _table = {}
    
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super(LeaderBoard, cls).__new__(cls)
        return cls.instance
    
    @classmethod
    def add_winner(cls, name: str):
        if name in cls._table:
            cls._table[name] += 1
        else:
            cls._table[name] = 1
            
    @classmethod
    def print(cls):
        print("🏆 Leaderboard 🏆")
        for name, score in cls._table.items():
            print(f"{name}: {score}")
            
            
class GameA(IGame):
    def __init__(self):
        self.leaderboard = LeaderBoard()
        
    def add_winner(self, name: str):
        self.leaderboard.add_winner(name)
        
    def get_leaderboard(self):
        self.leaderboard.print()
        
        
class GameB(IGame):
    def __init__(self):
        self.leaderboard = LeaderBoard()
        
    def add_winner(self, name):
        self.leaderboard.add_winner(name)
        
    def get_leaderboard(self):
        self.leaderboard.print()
        
        
gameA = GameA()
gameA.add_winner("Joyce")
gameA.get_leaderboard()

gameB = GameB()
gameB.add_winner("Greene")
gameB.get_leaderboard()

🏆 Leaderboard 🏆
Joyce: 1
🏆 Leaderboard 🏆
Joyce: 1
Greene: 1


In [6]:
gameB.get_leaderboard()

🏆 Leaderboard 🏆
Joyce: 1
Greene: 1
